<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-5/Session_5_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [27]:
import tensorflow as tf
print(tf.__version__)

if tf.__version__.startswith('2')==False :
  !pip uninstall tensorflow
  !pip install tensorflow-gpu
  print(tf.__version__)


2.0.0


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import xception, inception_v3, resnet_v2, vgg19,densenet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [0]:
# data = > https://drive.google.com/file/d/1GEKK8oRNntFyR0ZxPdcvPut-15b7CvrW/view?usp=sharing
# small Data => https://drive.google.com/file/d/1OHGNsTfvVZvWYQ7B29SYcxrLGVdeCoQb/view?usp=sharing




In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
if not os.path.exists('MLIntroData'):
    os.makedirs('MLIntroData')

# Download Zip
myzip = drive.CreateFile({'id': '1GEKK8oRNntFyR0ZxPdcvPut-15b7CvrW'})
myzip.GetContentFile('data.zip')

# 3. Unzip
zip_ref = zipfile.ZipFile('data.zip', 'r')
zip_ref.extractall('MLIntroData/data')
zip_ref.close()

In [31]:
if os.path.exists('MLIntroData'):
    print(os.listdir("MLIntroData/data/data"))


['.gitignore', 'test', 'train']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)

epochs = 100
x_train = []
y_train = []
y_labels = []

In [33]:
#define the sub folders for both training and test
training = os.path.join("MLIntroData/data/data",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=xception.preprocess_input,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          fill_mode='nearest')


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           shuffle=False
                                                           )

Found 229 images belonging to 5 classes.


In [34]:
y_train =  train_generator.classes
for k in train_generator.class_indices.keys():
    y_labels.append(k)
y_train = to_categorical(y_train)

print(len(y_train))

229


In [35]:
# NOW WE LOAD THE PRE_TRAINED MODEL
FEATURE_EXTRACTOR = vgg19.VGG19(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model = Sequential()
model.add(FEATURE_EXTRACTOR)
model.add(Flatten())
features_x = model.predict_generator(train_generator)
print(features_x.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(229, 4608)


In [36]:
FEATURE_EXTRACTOR1 = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model1 = Sequential()
model1.add(FEATURE_EXTRACTOR1)
model1.add(Flatten())
features_x1 = model1.predict_generator(train_generator)
print(features_x1.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(229, 18432)


In [37]:
FEATURE_EXTRACTOR2 = resnet_v2.ResNet152V2(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model2 = Sequential()
model2.add(FEATURE_EXTRACTOR2)
model2.add(Flatten())
features_x2 = model2.predict_generator(train_generator)
print(features_x2.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(229, 32768)


In [38]:
FEATURE_EXTRACTOR3 = inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model3 = Sequential()
model3.add(FEATURE_EXTRACTOR3)
model3.add(Flatten())
features_x3 = model3.predict_generator(train_generator)
print(features_x3.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(229, 2048)


In [39]:
FEATURE_EXTRACTOR4 = densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model4 = Sequential()
model4.add(FEATURE_EXTRACTOR4)
model4.add(Flatten())
features_x4 = model4.predict_generator(train_generator)
print(features_x4.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(229, 17280)


In [40]:
import numpy as np
all_features = np.concatenate((features_x, features_x1,features_x2,features_x3,features_x4), axis=1)
print(all_features.shape)

(229, 75136)


In [41]:
model = Sequential()

#add our layers
model.add(Flatten(input_shape=all_features.shape[1:]))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.2))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.5))
model.add(Dense(len(y_labels),activation='softmax'))
history = model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_24 (Flatten)         (None, 75136)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               9617536   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 645       
Total params: 9,634,693
Trainable params: 9,634,693
Non-trainable params: 0
___________________________________________

In [42]:
model.fit(all_features,y_train,epochs=epochs,shuffle=True,verbose=2)

Train on 229 samples
Epoch 1/100
229/229 - 1s - loss: 5.5271 - accuracy: 0.4760
Epoch 2/100
229/229 - 1s - loss: 2.3206 - accuracy: 0.7686
Epoch 3/100
229/229 - 1s - loss: 1.6017 - accuracy: 0.8690
Epoch 4/100
229/229 - 1s - loss: 0.6767 - accuracy: 0.9170
Epoch 5/100
229/229 - 1s - loss: 0.7127 - accuracy: 0.9301
Epoch 6/100
229/229 - 1s - loss: 0.7467 - accuracy: 0.9258
Epoch 7/100
229/229 - 1s - loss: 0.5341 - accuracy: 0.9432
Epoch 8/100
229/229 - 1s - loss: 0.7871 - accuracy: 0.9432
Epoch 9/100
229/229 - 1s - loss: 0.3140 - accuracy: 0.9651
Epoch 10/100
229/229 - 1s - loss: 1.0243 - accuracy: 0.9432
Epoch 11/100
229/229 - 1s - loss: 0.2303 - accuracy: 0.9694
Epoch 12/100
229/229 - 1s - loss: 0.1764 - accuracy: 0.9738
Epoch 13/100
229/229 - 1s - loss: 0.2312 - accuracy: 0.9651
Epoch 14/100
229/229 - 1s - loss: 0.1542 - accuracy: 0.9825
Epoch 15/100
229/229 - 1s - loss: 0.1354 - accuracy: 0.9825
Epoch 16/100
229/229 - 1s - loss: 0.2553 - accuracy: 0.9694
Epoch 17/100
229/229 - 1s - 

In [44]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("MLIntroData/data/data/test",
                                                         target_size=(100,100),
                                                         shuffle=False)

Found 93 images belonging to 5 classes.


In [45]:
FEATURE_EXTRACTOR =  vgg19.VGG19(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model0 = Sequential()
model0.add(FEATURE_EXTRACTOR)
model0.add(Flatten())
features_x = model0.predict_generator(test_generator)
print(type(features_x).__name__)
print(features_x.shape)

FEATURE_EXTRACTOR1 = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model1 = Sequential()
model1.add(FEATURE_EXTRACTOR1)
model1.add(Flatten())
features_x1 = model1.predict_generator(test_generator)
print(type(features_x1).__name__)
print(features_x1.shape)

FEATURE_EXTRACTOR2 = resnet_v2.ResNet152V2(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model2 = Sequential()
model2.add(FEATURE_EXTRACTOR2)
model2.add(Flatten())
features_x2 = model2.predict_generator(test_generator)
print(type(features_x2).__name__)
print(features_x2.shape)

FEATURE_EXTRACTOR3 = inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model3 = Sequential()
model3.add(FEATURE_EXTRACTOR3)
model3.add(Flatten())
features_x3 = model3.predict_generator(test_generator)
print(type(features_x3).__name__)
print(features_x3.shape)

FEATURE_EXTRACTOR4 = densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model4 = Sequential()
model4.add(FEATURE_EXTRACTOR4)
model4.add(Flatten())
features_x4 = model4.predict_generator(test_generator)
print(type(features_x4).__name__)
print(features_x4.shape)

all_features = np.concatenate((features_x, features_x1,features_x2,features_x3,features_x4), axis=1)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


ndarray
(93, 4608)
ndarray
(93, 18432)
ndarray
(93, 32768)
ndarray
(93, 2048)
ndarray
(93, 17280)


In [0]:
predictions = model.predict(all_features)

In [47]:
from sklearn.metrics import confusion_matrix,classification_report
row_index = predictions.argmax(axis=1)
filenames = test_generator.filenames
nb_samples = len(filenames)
y_true = test_generator.classes
target_names = test_generator.class_indices.keys()
print(target_names)
print(confusion_matrix(y_true, row_index))
print('Classification Report')
target_names = test_generator.class_indices.keys()
print(classification_report(test_generator.classes, row_index, target_names=target_names))

dict_keys(['bar_chart', 'bubble_chart', 'pie_chart', 'radar_chart', 'treemap_chart'])
[[18  1  0  0  0]
 [ 0 18  0  1  0]
 [ 0  1 16  1  0]
 [ 0  1  0 17  0]
 [ 0  0  0  0 19]]
Classification Report
               precision    recall  f1-score   support

    bar_chart       1.00      0.95      0.97        19
 bubble_chart       0.86      0.95      0.90        19
    pie_chart       1.00      0.89      0.94        18
  radar_chart       0.89      0.94      0.92        18
treemap_chart       1.00      1.00      1.00        19

     accuracy                           0.95        93
    macro avg       0.95      0.95      0.95        93
 weighted avg       0.95      0.95      0.95        93

